In [1]:
from copy import copy, deepcopy
from irace2 import irace, dummy_stats_test
import itertools
import numpy as np
from sampling_functions import norm_sample, truncated_poisson, truncated_skellam
import scipy.stats as stats
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split,StratifiedShuffleSplit,cross_val_score
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import scipy.stats as ss
from scipy.stats import norm, poisson, skellam
from tqdm import tqdm
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
Xs = []
ys = []

df = pd.read_csv('spect_train.csv')
Xs.append(preprocessing.normalize(df.drop(columns=['OVERALL_DIAGNOSIS']).to_numpy()))
ys.append(df['OVERALL_DIAGNOSIS'].to_numpy())

df = pd.read_csv('spambase.csv')
Xs.append(preprocessing.normalize(df.drop(columns=['spam']).to_numpy()))
ys.append(df['spam'].to_numpy())

df = pd.read_csv('ionosphere_data.csv')
Xs.append(preprocessing.normalize(df.drop(columns=['column_ai']).to_numpy()))
ys.append(df['column_ai'].to_numpy())

In [3]:
df = pd.read_csv('UCI_Credit_Card.csv')
Xs.append(preprocessing.normalize(df.drop(columns=['ID','default.payment.next.month']).to_numpy()))
ys.append(df['default.payment.next.month'].to_numpy())

In [4]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [5]:
df.shape

(30000, 25)

In [6]:
df = pd.read_csv('heart.csv')
Xs.append(preprocessing.normalize(df.drop(columns=['target']).to_numpy()))
ys.append(df['target'].to_numpy())

In [7]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [8]:
df.shape

(1025, 14)

In [9]:
for x in Xs:
    print(x.shape)

(80, 44)
(4601, 57)
(351, 34)
(30000, 23)
(1025, 13)


In [10]:
cross_val_score(LogisticRegression(), Xs[4][0:600], ys[4][0:600], cv=30, scoring='f1')

array([0.72727273, 0.6       , 0.64      , 0.8       , 0.59259259,
       0.60869565, 0.8       , 0.76923077, 0.76190476, 0.69565217,
       0.76190476, 0.71428571, 0.7826087 , 0.66666667, 0.73684211,
       0.7826087 , 0.58823529, 0.66666667, 0.7826087 , 0.72      ,
       0.72727273, 0.60869565, 0.7826087 , 0.7826087 , 0.7826087 ,
       0.78571429, 0.53846154, 0.75      , 0.86956522, 0.75      ])

In [12]:
df = pd.read_excel('impulsividade.xls')

In [13]:
df.head()

,sex,age,Copeptin,Angiotensin I,Angiotensin II,TFG-alpha,IFN-alpha 2,GRO,MCP-3,MDC,...,TNFSF14/LIGHT,RAGE,Enolase-2/NSE,NRG1-beta 1,BIS attentional,BIS motor,BIS nonplanning,BIS total,BIS 2F inhibitory control,BIS 2F nonplanning
0,1,37.0,70302,120200,26653,0.94,6.47,23020.0,0.04,613.37,...,34.04,2084.0,1818.00,10.91,17,19,22,58,39.0,14.0
1,0,42.0,72694,70954,63925,3.82,0.00,1855.0,0.01,917.21,...,192.70,1817.0,7959.00,7.08,11,20,28,59,35.0,19.0
2,0,37.0,80197,90722,68961,3.82,0.12,9737.0,0.01,1615.00,...,179.92,2271.0,2393.00,10.91,23,20,24,67,42.0,18.0
3,1,25.0,88385,92798,72021,3.63,11.83,4945.0,12.44,1354.00,...,37.09,1163.0,671.57,8.10,25,24,30,79,56.0,19.0
4,1,32.0,68522,156122,36780,3.82,3.37,2025.0,0.01,894.43,...,65.35,2271.0,6152.00,10.91,18,21,26,65,39.0,20.0


In [1]:
parameters_dict = {
    'LogisticRegression': {'C': lambda loc : norm_sample(loc=loc, scale=1, min= 1e-2),
                            'penalty':['l2'],
                            'solver':['lbfgs','newton-cg','sag']},
    'SVC':{'C':lambda loc : norm_sample(loc=loc, scale=1, min= 1e-2),
            'coef0': lambda loc : norm_sample(loc=loc, scale=1, min= 1e-2),
            'kernel':['linear','poly','rbf','sigmoid'],
            'decision_function_shape':['ovo','ovr']},
    'RandomForestClassifier': {'n_estimators': lambda loc: truncated_skellam(loc, mu1=10, mu2=10, min=1), 
                                'max_depth': lambda loc: truncated_skellam(loc, mu1=1, mu2=1, min=1),
                                'max_features':['sqrt', 'log2', None],
                                'ccp_alpha':lambda loc : norm_sample(loc=loc, scale=0.1, min= 1e-3)
                                },
    'XGBClassifier': {'sample_type': ['uniform','weighted'], 
                        'max_depth': lambda loc: truncated_skellam(loc, mu1=1, mu2=1, min=1),
                        'booster':['gbtree','dart'],
                        'subsample':lambda loc : norm_sample(loc=loc, scale=0.3, min= 1e-2,max=1)}
}

In [2]:
par = {}

In [4]:
for p in parameters_dict:
    par.update({p:10})

In [5]:
par

{'LogisticRegression': 10,
 'SVC': 10,
 'RandomForestClassifier': 10,
 'XGBClassifier': 10}